In [1]:
import os
import requests
import json
from datetime import datetime

import pandas as pd
import numpy as np

In [9]:
# Archive previous dataset
old_datas = [dir for dir in os.listdir() if "steam" in dir]
for old_data in old_datas:
    os.rename(old_data, "old_datasets/" + old_data)    
    
# Get first 1000 more popular Steam Games by concurrent users
res = requests.get("https://steamspy.com/api.php?request=all&page=0")

# Deserialize
data = json.loads(res.text)

# Iterate over appids to get more detailed info per game
n_rows = len(data.keys())
rows = []
for idx, appid in enumerate(data.keys()):
    row = json.loads(
        requests
        .get("https://steamspy.com/api.php?request=appdetails&appid={}".format(appid))
        .text
    )
    
    if row['tags']:
        row['tags'] = ";".join(row['tags'].keys())
    else:
        row['tags'] = "None"
    
    if row['genre']:
        row['genre'] = ";".join(row['genre'].split(", "))
    else:
        row['genre'] = "None"
    
    rows.append(row)
    print('\r', 'Progress:', str(round((idx / (n_rows-1)) * 100, 2)) + "%", 'Appid', appid, end='')

df = pd.DataFrame(rows)

# Store data
name = "df_steam_" + datetime.now().strftime("%d-%m-%Y_%H-%M") + ".csv"
df.to_csv(name, index = False)

tags = pd.DataFrame(np.unique(";".join(df["tags"]).split(";")), columns = ['tag'])
genres = pd.DataFrame(np.unique(";".join(df["genre"]).split(";")), columns = ['genre'])

tags.to_csv("tags.csv", index = False)
genres.to_csv("genres.csv", index = False)

df.head()

 Progress: 100.0% Appid 2903400

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu,languages,genre,tags
0,570,Dota 2,Valve,Valve,,1620812,341671,0,"200,000,000 .. 500,000,000",38131,1313,879,730,0,0,0,520119,"English, Bulgarian, Czech, Danish, Dutch, Finn...",Action;Free to Play;Strategy,Free to Play;MOBA;Multiplayer;Strategy;e-sport...
1,730,Counter-Strike: Global Offensive,"Valve, Hidden Path Entertainment",Valve,,6317781,810361,0,"50,000,000 .. 100,000,000",30485,759,5864,286,0,0,0,1223076,"English, Czech, Danish, Dutch, Finnish, French...",Action;Free to Play,FPS;Shooter;Multiplayer;Competitive;Action;Tea...
2,1172470,Apex Legends,Respawn Entertainment,Electronic Arts,,523297,109790,0,"50,000,000 .. 100,000,000",7186,809,813,419,0,0,0,417985,"English, French, Italian, German, Spanish - Sp...",Action;Adventure;Free to Play,Free to Play;Multiplayer;Battle Royale;Shooter...
3,578080,PUBG: BATTLEGROUNDS,"KRAFTON, Inc.","KRAFTON, Inc.",,1237022,927771,0,"50,000,000 .. 100,000,000",22055,691,6340,188,0,0,0,381064,"English, Korean, Simplified Chinese, French, G...",Action;Adventure;Free to Play;Massively Multip...,Survival;Shooter;Battle Royale;Multiplayer;FPS...
4,1063730,New World,Amazon Games,Amazon Games,,177176,76296,0,"50,000,000 .. 100,000,000",7386,338,3225,167,3999,3999,0,23870,"English, French, Italian, German, Spanish - Sp...",Action;Adventure;Massively Multiplayer;RPG,Massively Multiplayer;Open World;MMORPG;RPG;Ad...
